In [ ]:
# Install Libraries and create output folders

import sys
!{sys.executable} -m pip install tifffile
!{sys.executable} -m pip install opencv-python-headless
!{sys.executable} -m pip install keras==2.2.4
!{sys.executable} -m pip install imutils
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install scikit-image
!{sys.executable} -m pip install rasterio
import os
try:
    os.mkdir("logs")
except:
    pass
try:
    os.mkdir("weights")
except:
    pass

In [ ]:
# Import libraries

import glob
import random
import sys
import math
from random import randint

import random
random.seed(1337)
import numpy as np
np.random.seed(1337)
import tensorflow as tf
tf.set_random_seed(1337)

import cv2
import imutils
import keras
import matplotlib.pyplot as plt

import tifffile as tiff
import rasterio
from keras import backend
from tensorflow import ones_like, equal, log
from trainvaltensorboard import TrainValTensorBoard
from keras.callbacks import ModelCheckpoint

from keras.layers import Input, Conv2D, AlphaDropout, MaxPooling2D, Conv2DTranspose, BatchNormalization, Activation
from keras.layers.merge import concatenate
from keras.metrics import mean_squared_error
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from matplotlib.pyplot import figure
from scipy import ndimage, misc
from skimage.color import rgb2gray

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50

In [ ]:
# Load Planet images

img_labels_in_30 = tiff.imread("binary_30.tif")
img_labels_in_27 = tiff.imread("binary_27.tif")
img_labels_in_23 = tiff.imread("binary_23.tif")
img_labels_in_30 = img_labels_in_30[:,:,3] == 255
img_labels_in_27 = img_labels_in_27[:,:,3] == 255
img_labels_in_23 = img_labels_in_23[:,:,3] == 255
img_labels_in_30 = img_labels_in_30[0:3334,0:5500] == 0
img_labels_in_27 = img_labels_in_27[0:3334,0:5500] == 0
img_labels_in_23 = img_labels_in_23[0:3334,0:5500] == 0

img_data_in_23 = rasterio.open("Mosaic_20170823_DHDN_clip.tif")
img_data_in_23 = np.concatenate((np.expand_dims(img_data_in_23.read(1), axis=2),np.expand_dims(img_data_in_23.read(2), axis=2),np.expand_dims(img_data_in_23.read(3), axis=2),np.expand_dims(img_data_in_23.read(4), axis=2)), axis=2)
img_data_in_27 = rasterio.open("Mosaic_20170827_DHDN_clip.tif")
img_data_in_27 = np.concatenate((np.expand_dims(img_data_in_27.read(1), axis=2),np.expand_dims(img_data_in_27.read(2), axis=2),np.expand_dims(img_data_in_27.read(3), axis=2),np.expand_dims(img_data_in_27.read(4), axis=2)), axis=2)
img_data_in_30 = rasterio.open("Mosaic_20170830_DHDN_clip.tif")
img_data_in_30 = np.concatenate((np.expand_dims(img_data_in_30.read(1), axis=2),np.expand_dims(img_data_in_30.read(2), axis=2),np.expand_dims(img_data_in_30.read(3), axis=2),np.expand_dims(img_data_in_30.read(4), axis=2)), axis=2)
img_labels_in = np.concatenate((np.expand_dims(img_labels_in_30, axis=0), np.expand_dims(img_labels_in_27, axis=0), np.expand_dims(img_labels_in_23, axis=0)), axis=0)
img_data_in = np.concatenate((np.expand_dims(img_data_in_30[0:3334,0:5500,:], axis=0), np.expand_dims(img_data_in_27[0:3334,0:5500,:], axis=0), np.expand_dims(img_data_in_23[0:3334,0:5500,:], axis=0)), axis=0)
img_data_in = (img_data_in / 65536).astype(np.float32)

In [ ]:
# Load Ortho tiles

val_data = [
    "4606000-5396000",
    "4608000-5390000",
    "4610000-5394000",
    "4614000-5392000",
    "4614000-5388000",
    "4616000-5394000",
    "4618000-5390000",
    "4622000-5392000"
]
data = glob("Orthos/*/")
train_data = []
for d in data:
    if(d.split("/")[1] not in val_data):
        train_data.append(d)
data_tiles_list = []
label_tiles_list = []
for p in train_data:
    data_tiles_list.append(glob(p + "/images/*.tif"))
    label_tiles_list.append(glob(p + "/labels/0/*.tif"))

data_tiles_list = [y for x in data_tiles_list for y in x]
label_tiles_list = [y for x in label_tiles_list for y in x]

training_images = []
for i in range(len(data_tiles_list)):
    print(str(i) + "/" + str(len(data_tiles_list)))
    data_in = rasterio.open(data_tiles_list[i])
    training_images.append(np.concatenate((np.expand_dims(data_in.read(1), axis=2),np.expand_dims(data_in.read(2), axis=2),np.expand_dims(data_in.read(3), axis=2),np.expand_dims(data_in.read(4), axis=2)), axis=2))
training_images = np.asarray(training_images)
training_images = (training_images / 256).astype(np.float32)
print(training_images.shape)

training_labels = []
for i in range(len(label_tiles_list)):
    print(str(i) + "/" + str(len(label_tiles_list)))
    data_in = data_in = rasterio.open(label_tiles_list[i])
    training_labels.append(data_in.read(1))
training_labels = np.asarray(training_labels)
training_labels = np.expand_dims(training_labels, axis=3)
training_labels = training_labels == 0
print(training_labels.shape)

In [ ]:
# Load Ortho tiles without damage and create empty labeling, then merge the tiles with the Ortho tiles

data = glob("Orthos NoDamage/*/")
train = []
data_tiles_list_s = []
label_tiles_list_s = []
for p in data:
    data_tiles_list_s.append(glob(p + "/images/*.tif"))
    label_tiles_list_s.append(glob(p + "/labels/0/*.tif"))

data_tiles_list_s = [y for x in data_tiles_list_s for y in x]
label_tiles_list_s = [y for x in label_tiles_list_s for y in x]

training_images_s = []
for i in range(len(data_tiles_list_s)):
    print(str(i) + "/" + str(len(data_tiles_list_s)))
    data_in = rasterio.open(data_tiles_list_s[i])
    training_images_s.append(np.concatenate((np.expand_dims(data_in.read(1), axis=2),np.expand_dims(data_in.read(2), axis=2),np.expand_dims(data_in.read(3), axis=2),np.expand_dims(data_in.read(4), axis=2)), axis=2))
training_images_s = np.asarray(training_images_s)
training_images_s = (training_images_s / 256).astype(np.float32)
print(training_images_s.shape)

training_labels_s = []
for i in range(training_images_s.shape[0]):
    t = np.zeros((256,256))
    training_labels_s.append(t)
training_labels_s = np.expand_dims(np.asarray(training_labels_s), axis=3)
training_labels_s = training_labels_s == 0
print(training_labels_s.shape)

training_images_s2 = training_images_s
training_labels_s2 = training_labels_s

training_images = np.concatenate((training_images, training_images_s2), axis=0)
training_labels = np.concatenate((training_labels, training_labels_s2), axis=0)

In [ ]:
# Display Planet label image

figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(img_labels_in[0,:,:].astype(np.uint8), cmap="gray")
plt.xticks([])
plt.yticks([])

In [ ]:
# Display Planet mage

figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
plt.imshow((img_data_in_noDamage[0][:,:,0:3]*256).astype(np.uint8)*6)
plt.xticks([])
plt.yticks([])

In [ ]:
# Planet Image and label tiling, then data augmentation

rotation = [1,3] # Rot 180 can be produced by HOR+VER Flip!
flip_horizontal = True
flip_vertical = True
validation_split = 0.235 # fit 3 rows of test at bottom
validation_split = 0.154 # fit 2 rows of test at bottom
tile_size_x = 256
tile_size_y = 256
stride_x = 128
stride_y = 128
random_select = False
num_color_images = 0
sigma = 1.5

split_height = int(img_data_in.shape[1] * validation_split)
img_data_in_d = img_data_in[:,:-split_height,:,:]
img_labels_in_d = img_labels_in[:,:-split_height,:]
img_data_in_v = img_data_in[:,-split_height:,:,:]
img_labels_in_v = img_labels_in[:,-split_height:,:]
print("Validation Split: " + str(img_data_in_d.shape[1]) + "/" + str(img_data_in_v.shape[1]))

labels_training = []
images_training = []
for o in range(img_labels_in_d.shape[0]):
    i = 0
    while i <= img_labels_in_d[o].shape[0] - tile_size_y:
        j = 0
        while j <= img_labels_in_d[o].shape[1] - tile_size_x:
            tile = img_labels_in_d[o, i:i+tile_size_y, j:j+tile_size_x]
            if np.sum(tile) > 0:
                tile2 = img_data_in_d[o, i:i+tile_size_y, j:j+tile_size_x]
                if np.min(tile2 > 0):
                    labels_training.append(tile)
                    images_training.append(tile2)
            j += tile_size_x - stride_x
        i += tile_size_y - stride_y
    
training_labels = np.expand_dims(np.asarray(labels_training), axis=3) == 0
training_images = np.asarray(images_training)

labels_validation = []
images_validation = []
for o in range(img_labels_in_v.shape[0]):
    i = 0
    while i <= img_labels_in_v[o].shape[0] - tile_size_y:
        j = 0
        while j <= img_labels_in_v[o].shape[1] - tile_size_x:
            tile = img_labels_in_v[o, i:i+tile_size_y, j:j+tile_size_x]
            if np.sum(tile) > 0:
                tile2 = img_data_in_v[o, i:i+tile_size_y, j:j+tile_size_x]
                if np.min(tile2 > 0):
                    labels_validation.append(tile)
                    images_validation.append(tile2)
            j += tile_size_x# - stride_x
        i += tile_size_y# - stride_y

validation_labels = np.expand_dims(np.asarray(labels_validation), axis=3) == 0
validation_images = np.asarray(images_validation)

print("Number of training labels/images: " + str(training_labels.shape))
print("Number of validation labels/images: " + str(validation_labels.shape))

if str(random_select).isdigit():
    random_select_split = int(training_images.shape[0] * random_select)
    o = training_images.shape[0]
    training_images = training_images[:random_select_split,:,:,:]
    training_labels = training_labels[:random_select_split,:,:,:]
    print("Random select tiles: " + str(training_images.shape[0]) + " of " + str(o))

if len(rotation) > 0:
    training_images_rot = np.copy(training_images)
    training_labels_rot = np.copy(training_labels)
    for i in rotation:
        training_images_rot = np.concatenate((training_images_rot, np.rot90(training_images, i, (1,2))), axis=0)
        training_labels_rot = np.concatenate((training_labels_rot, np.rot90(training_labels, i, (1,2))), axis=0)
    training_images = training_images_rot
    training_labels = training_labels_rot
    print("After rotation:")
    print("Number of training labels/images: " + str(training_labels.shape))

if flip_horizontal:
    training_images = np.concatenate((training_images, np.flip(training_images, 2)), axis=0)
    training_labels = np.concatenate((training_labels, np.flip(training_labels, 2)), axis=0)
    print("After horizontal flip:")
    print("Number of training labels/images: " + str(training_labels.shape))

if flip_vertical:
    training_images = np.concatenate((training_images, np.flip(training_images, 1)), axis=0)
    training_labels = np.concatenate((training_labels, np.flip(training_labels, 1)), axis=0)
    print("After vertical flip:")
    print("Number of training labels/images: " + str(training_labels.shape))
    
training_data = list(zip(training_images, training_labels))
random.shuffle(training_data)
training_images[:], training_labels[:] = zip(*training_data)

if num_color_images > -1:
    training_images6 = np.copy(training_images)
    training_labels6 = np.copy(training_labels)
    training_images3 = np.array(training_images)
    training_labels3 = np.array(training_labels)
    for i in range(num_color_images):
        training_images3 = np.concatenate((training_images3, training_images), axis=0)
        training_labels3 = np.concatenate((training_labels3, training_labels), axis=0)
    training_images = training_images3
    training_labels = training_labels3
    for img in training_images:
        img[:,:,0] = img[:,:,0] + np.random.normal(0, sigma) / 256
        img[:,:,0] = np.where(img[:,:,0] < 1, img[:,:,0], 1)
        img[:,:,0] = np.where(img[:,:,0] >= 0, img[:,:,0], 0)
        img[:,:,1] = img[:,:,1] + np.random.normal(0, sigma) / 256
        img[:,:,1] = np.where(img[:,:,1] < 1, img[:,:,1], 1)
        img[:,:,1] = np.where(img[:,:,1] >= 0, img[:,:,1], 0)
        img[:,:,2] = img[:,:,2] + np.random.normal(0, sigma) / 256
        img[:,:,2] = np.where(img[:,:,2] < 1, img[:,:,2], 1)
        img[:,:,2] = np.where(img[:,:,2] >= 0, img[:,:,2], 0)
        img[:,:,3] = img[:,:,3] + np.random.normal(0, sigma) / 256
        img[:,:,3] = np.where(img[:,:,3] < 1, img[:,:,3], 1)
        img[:,:,3] = np.where(img[:,:,3] >= 0, img[:,:,3], 0)
    training_images = np.concatenate((training_images, training_images6), axis=0)
    training_labels = np.concatenate((training_labels, training_labels6), axis=0)
    print("After color variations:")
    print("Number of training labels/images: " + str(training_labels.shape))

In [ ]:
# Ortho data augmentation

rotation = [1]
flip_horizontal = True
flip_vertical = False
random_select = 1
num_color_images = 1
sigma = 1.5

print("Number of training labels/images: " + str(training_labels.shape))

if len(rotation) > 0:
    training_images_rot = np.copy(training_images)
    training_labels_rot = np.copy(training_labels)
    for i in rotation:
        training_images_rot = np.concatenate((training_images_rot, np.rot90(training_images, i, (1,2))), axis=0)
        training_labels_rot = np.concatenate((training_labels_rot, np.rot90(training_labels, i, (1,2))), axis=0)
    training_images = training_images_rot
    training_labels = training_labels_rot
    print("After rotation:")
    print("Number of training labels/images: " + str(training_labels.shape))

if flip_horizontal:
    training_images = np.concatenate((training_images, np.flip(training_images, 2)), axis=0)
    training_labels = np.concatenate((training_labels, np.flip(training_labels, 2)), axis=0)
    print("After horizontal flip:")
    print("Number of training labels/images: " + str(training_labels.shape))

if flip_vertical:
    training_images = np.concatenate((training_images, np.flip(training_images, 1)), axis=0)
    training_labels = np.concatenate((training_labels, np.flip(training_labels, 1)), axis=0)
    print("After vertical flip:")
    print("Number of training labels/images: " + str(training_labels.shape))
    
training_data = list(zip(training_images, training_labels))
random.shuffle(training_data)
training_images[:], training_labels[:] = zip(*training_data)

if random_select > 0:
    random_select_split = int(training_images.shape[0] * random_select)
    training_images = training_images[:random_select_split,:,:,:]
    training_labels = training_labels[:random_select_split,:,:,:]
    print("Random select tiles: " + str(training_images.shape[0]) + " of " + str(o))
    
if num_color_images > -1:
    training_images6 = np.copy(training_images)
    training_labels6 = np.copy(training_labels)
    training_images3 = np.array(training_images)
    training_labels3 = np.array(training_labels)
    for i in range(num_color_images):
        training_images3 = np.concatenate((training_images3, training_images), axis=0)
        training_labels3 = np.concatenate((training_labels3, training_labels), axis=0)
    training_images = training_images3
    training_labels = training_labels3
    for img in training_images:
        img[:,:,0] = img[:,:,0] + np.random.normal(0, sigma) / 256
        img[:,:,0] = np.where(img[:,:,0] < 1, img[:,:,0], 1)
        img[:,:,0] = np.where(img[:,:,0] >= 0, img[:,:,0], 0)
        img[:,:,1] = img[:,:,1] + np.random.normal(0, sigma) / 256
        img[:,:,1] = np.where(img[:,:,1] < 1, img[:,:,1], 1)
        img[:,:,1] = np.where(img[:,:,1] >= 0, img[:,:,1], 0)
        img[:,:,2] = img[:,:,2] + np.random.normal(0, sigma) / 256
        img[:,:,2] = np.where(img[:,:,2] < 1, img[:,:,2], 1)
        img[:,:,2] = np.where(img[:,:,2] >= 0, img[:,:,2], 0)
        img[:,:,3] = img[:,:,3] + np.random.normal(0, sigma) / 256
        img[:,:,3] = np.where(img[:,:,3] < 1, img[:,:,3], 1)
        img[:,:,3] = np.where(img[:,:,3] >= 0, img[:,:,3], 0)
    training_images = np.concatenate((training_images, training_images6), axis=0)
    training_labels = np.concatenate((training_labels, training_labels6), axis=0)
    print("After color variations:")
    print("Number of training labels/images: " + str(training_labels.shape))

In [ ]:
# Display some Planet images

figure(num=None, figsize=(16, 8), dpi=80, facecolor='w', edgecolor='k')
r = randint(0, training_labels.shape[0] - 1)
print(r)
plt.subplot(2, 4, 1)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int)*6)
plt.xticks([])
plt.yticks([])
plt.subplot(2, 4, 2)
plt.imshow(training_labels[r,:,:,0])
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
plt.subplot(2, 4, 3)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int)*6)
plt.xticks([])
plt.yticks([])
plt.subplot(2, 4, 4)
plt.imshow(training_labels[r,:,:,0])
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
plt.subplot(2, 4, 5)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int)*6)
plt.xticks([])
plt.yticks([])
plt.subplot(2, 4, 6)
plt.imshow(training_labels[r,:,:,0])
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
plt.subplot(2, 4, 7)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int)*6)
plt.xticks([])
plt.yticks([])
plt.subplot(2, 4, 8)
plt.imshow(training_labels[r,:,:,0])
plt.xticks([])
plt.yticks([])

In [ ]:
# Display some Ortho images

figure(num=None, figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
r = randint(0, training_labels.shape[0] - 1)
print(r)
print(np.min(training_labels[r,:,:,0]))
print(np.max(training_labels[r,:,:,0]))
print()
plt.subplot(3, 4, 1)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int))
plt.xticks([])
plt.yticks([])
plt.subplot(3, 4, 2)
plt.imshow(training_labels[r,:,:,0], cmap="gray")
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
print(np.min(training_labels[r,:,:,0]))
print(np.max(training_labels[r,:,:,0]))
print()
plt.subplot(3, 4, 3)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int))
plt.xticks([])
plt.yticks([])
plt.subplot(3, 4, 4)
plt.imshow(training_labels[r,:,:,0], cmap="gray")
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
print(np.min(training_labels[r,:,:,0]))
print(np.max(training_labels[r,:,:,0]))
print()
plt.subplot(3, 4, 5)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int))
plt.xticks([])
plt.yticks([])
plt.subplot(3, 4, 6)
plt.imshow(training_labels[r,:,:,0], cmap="gray")
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
print(np.min(training_labels[r,:,:,0]))
print(np.max(training_labels[r,:,:,0]))
print()
plt.subplot(3, 4, 7)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int))
plt.xticks([])
plt.yticks([])
plt.subplot(3, 4, 8)
plt.imshow(training_labels[r,:,:,0], cmap="gray")
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
print(np.min(training_labels[r,:,:,0]))
print(np.max(training_labels[r,:,:,0]))
print()
plt.subplot(3, 4, 9)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int))
plt.xticks([])
plt.yticks([])
plt.subplot(3, 4, 10)
plt.imshow(training_labels[r,:,:,0], cmap="gray")
plt.xticks([])
plt.yticks([])
r = randint(0, training_labels.shape[0] - 1)
print(r)
print(np.min(training_labels[r,:,:,0]))
print(np.max(training_labels[r,:,:,0]))
print()
plt.subplot(3, 4, 11)
plt.imshow((training_images[r,:,:,0:3]*256).astype(int))
plt.xticks([])
plt.yticks([])
plt.subplot(3, 4, 12)
plt.imshow(training_labels[r,:,:,0], cmap="gray")
plt.xticks([])
plt.yticks([])

In [ ]:
# Create the model

bands = training_images.shape[3]
image_height = training_images.shape[1]
image_width = training_images.shape[2]

if backend.image_data_format() == 'channels_first':
    channel_axis = 1
    input_shape = (bands, image_height, image_width)
if backend.image_data_format() == 'channels_last':
    channel_axis = 3
    input_shape = (image_height, image_width, bands)
inp = Input(input_shape)

filter_size = (3, 3)
blocks = [8,16,32,64,128]
blocks2 = blocks
activation = 'relu'
filter_initializer = 'lecun_normal'

#encoder
encoder = inp
encoder_list = []
for block_id , n_block in enumerate(blocks):
    with backend.name_scope('Encoder_block{0}'.format(block_id)):
        encoder = Conv2D(filters = n_block, kernel_size = filter_size, activation = None, padding = 'same', kernel_initializer = filter_initializer) (encoder)
        encoder = BatchNormalization(axis=channel_axis, momentum=0.9) (encoder)
        encoder = Activation(activation) (encoder)
        encoder = AlphaDropout(0, 1*block_id) (encoder)
        encoder = Conv2D(filters = n_block, kernel_size = filter_size, dilation_rate = (2, 2), activation = None, padding='same', kernel_initializer = filter_initializer) (encoder)
        encoder = BatchNormalization(axis=channel_axis, momentum=0.9) (encoder)
        encoder = Activation(activation) (encoder)
        encoder_list.append(encoder)
        
        #maxpooling between every 2 blocks
        if block_id < len(blocks) - 1:
            encoder = MaxPooling2D(pool_size = (2, 2)) (encoder)
            
#decoder
decoder = encoder
blocks = blocks[1:]
for block_id, n_block in enumerate(blocks):
    with backend.name_scope('Decoder_block_{0}'.format(block_id)):
        block_id_inv = len(blocks) - block_id
        decoder = concatenate([decoder, encoder_list[block_id_inv]], axis = channel_axis)
        decoder = AlphaDropout(0, 1*block_id) (decoder)
        decoder = Conv2D(filters = n_block, kernel_size = filter_size, activation = None, padding = 'same', kernel_initializer = filter_initializer) (decoder)
#         decoder = BatchNormalization(axis=channel_axis, momentum=0.9) (decoder)
        decoder = Activation(activation) (decoder)
        decoder = Conv2DTranspose(filters = n_block, kernel_size = filter_size, kernel_initializer = filter_initializer, padding = 'same', strides=(2, 2)) (decoder)

outp = Conv2DTranspose(filters=1, kernel_size = filter_size, activation = 'sigmoid', padding = 'same', kernel_initializer = keras.initializers.glorot_normal(seed=1337)) (decoder)
model = Model(inputs = inp, outputs = outp)
print("Model Layers: " + str(int((len(model.layers)+1)/2)))

In [ ]:
# Compile the model

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 1.0):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(labels=y_true,predictions = y_pred_, num_classes = 2, weights = y_true) # Confusion matrix of [num_classes, num_classes]
        backend.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return backend.mean(backend.stack(prec), axis=0)

def create_weighted_binary_crossentropy(zero_weight, one_weight):
    def weighted_binary_crossentropy(y_true, y_pred):
        # Original binary crossentropy (see losses.py):
        # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)

        # Calculate the binary crossentropy
        b_ce = backend.binary_crossentropy(y_true, y_pred)

        # Apply the weights
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce
        
        # Return the mean error
        return backend.mean(weighted_b_ce)
    return weighted_binary_crossentropy

lr = 0.002
optimizer = keras.optimizers.Adam(lr = lr)
l1 = round(np.sum(training_labels) / (training_labels.shape[0] * training_labels.shape[1] * training_labels.shape[2]), 4)
l2 = round(1 - l1, 4)
loss = create_weighted_binary_crossentropy(l1, l2)
metrics = [mean_iou]
num_gpus = 1

if num_gpus > 1:
    self.model = multi_gpu_model(self.model, gpus=num_gpus)

model.compile(loss = loss, optimizer = optimizer, metrics = metrics)
model.name = "MODEL"

print(model.summary())

def_str = "Ortho_NB2_" + str(256) + "_" + str(blocks2) + "_" + str(lr) + "_" + str(num_color_images) + "-" + str(sigma) + "_loss_" + str(l1) + "_" + str(l2)
print(def_str)

tensorboard = TrainValTensorBoard(log_dir = "logs/" + def_str)
filepath = "weights/" + def_str + "-{epoch:02d}-{val_mean_iou:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_mean_iou', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
callback_list = [tensorboard, checkpoint]

In [ ]:
# Train the model

model.fit(training_images, training_labels, verbose=1, validation_split = 0.2, batch_size = 80, epochs = 15, initial_epoch = 0, callbacks = callback_list)

In [ ]:
# Load model weights from .hdf5 file

model.load_weights("final_NB2_256_[8, 16, 32, 64, 128]_0.002_0-1.5_loss_0.9434_0.0566-09-0.45.hdf5")

In [ ]:
# Save model to .h5 file

model.save(def_str + ".h5")

In [ ]:
# Load model from .h5 file

model = load_model("Ortho_NB2_256_[8, 16, 32, 64, 128]_0.002_1-1.5_loss_0.6701_0.3299-03-0.43.h5", custom_objects={'mean_iou': mean_squared_error, 'weighted_binary_crossentropy': create_weighted_binary_crossentropy(0.5, 0.5)})

In [ ]:
# Calculate the prediction

prediction = model.predict(validation_images, verbose=1, batch_size=1)

In [ ]:
# Calculate and plot metrics

print(prediction.shape)
val_labels = training_labels
print(val_labels.shape)
iou = []
tp = []
fp = []
tn = []
fn = []
err = []
accuracy = []
xaxis = []
for j in range(0, 100, 1):
    xaxis.append(j / 100)
    prediction2 = prediction > (j / 100)
    
    intersection = np.logical_and(prediction2 == 0, val_labels == 0)
    union = ((1 - prediction2) + (1 - val_labels)) > 0
    iou.append(np.sum(intersection) / np.sum(union))
    
    tn2 = np.sum(np.logical_and(prediction2 == val_labels, val_labels == 0))
    fn2 = np.sum(np.logical_and(prediction2 != val_labels, val_labels == 1))
    tp2 = np.sum(np.logical_and(prediction2 == val_labels, val_labels == 1))
    fp2 = np.sum(np.logical_and(prediction2 != val_labels, val_labels == 0))
    
    tn.append(tp2 / (tp2+fn2))
    fn.append(fp2 / (tn2+fp2))
    tp.append(tn2 / (tn2+fp2))
    fp.append(fn2 / (tp2+fn2))
    
    accuracy.append((tp2+tn2) / (tp2+tn2+fp2+fn2))
    err.append((fp2+fn2)/(tp2+tn2+fn2+fp2))
    print(j)

print("Max IoU: " + str(max(iou)) + ": " + str(iou.index(max(iou))))
print("Max TP: " + str(max(tp)) + ": " + str(tp.index(max(tp))))
print("Max FP: " + str(max(fp)) + ": " + str(fp.index(max(fp))))
print("Max TN: " + str(max(tn)) + ": " + str(tn.index(max(tn))))
print("Max FN: " + str(max(fn)) + ": " + str(fn.index(max(fn))))
print("Max Acc: " + str(max(accuracy)) + ": " + str(accuracy.index(max(accuracy))))
print("Max Err: " + str(max(err)) + ": " + str(err.index(max(err))))

figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
plt.plot(xaxis, iou, label="IoU")
plt.plot(xaxis, tp, label="True Positive Rate")
plt.plot(xaxis, fp, label="False Positive Rate")
plt.plot(xaxis, tn, label="True negative Rate")
plt.plot(xaxis, fn, label="False negative Rate")
plt.plot(xaxis, accuracy, label="Accuracy")
plt.plot(xaxis, err, label="Error Rate")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.axis([0, 1, 0, 1])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
# Crate ROC curve plot

figure(num=None, figsize=(10, 5), dpi=80, facecolor='w', edgecolor='k')
plt.plot(fp, tp)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.axis([0, 1, 0, 1])
plt.show()

In [ ]:
# Compress tensorboard logs folder in order to download it

import tarfile
import os
def recursive_files(dir_name='.', ignore=None):
    for dir_name,subdirs,files in os.walk(dir_name):
        if ignore and os.path.basename(dir_name) in ignore: 
            continue
        for file_name in files:
            if ignore and file_name in ignore:
                continue
            yield os.path.join(dir_name, file_name)
def make_tar_file(dir_name='.', tar_file_name='tarfile.tar', ignore=None):
    tar = tarfile.open(tar_file_name, 'w')
    for file_name in recursive_files(dir_name, ignore):
        tar.add(file_name)
    tar.close()
dir_name = 'logs'
tar_file_name = 'logs/logs.tar'
ignore = {'.ipynb_checkpoints', '__pycache__', tar_file_name}
make_tar_file(dir_name, tar_file_name, ignore)